In [1]:
import pandas as pd
from transformers import AutoTokenizer
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from utils import emoji_dict, stop_words, nouns, adjectives
from nltk.tokenize import RegexpTokenizer
import tensorflow as tf
import numpy as np


d:\COURSE\Computer science\Bangkit\Project\GradioProto\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = load_model('model/bert_attention_v6.h5')

model.summary()

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_16      │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_16        │ (None, 128, 768)  │ 24,516,864 │ input_layer_16[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_15    │ (None, 128, 256)  │    918,528 │ embedding_16[0][… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_14        │ (None, 128, 256)  │          0 │ bidirectional_15… │
│ (Attention)         │                   │            │ bidirectional_15… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ attention_14[0][… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_30 (Dense)    │ (None, 128)       │     32,896 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_24          │ (None, 128)       │          0 │ dense_30[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_31 (Dense)    │ (None, 3)         │        387 │ dropout_24[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 25,468,678 (97.16 MB)

 Trainable params: 951,811 (3.63 MB)

 Non-trainable params: 24,516,864 (93.52 MB)

 Optimizer params: 3 (16.00 B)

In [3]:
url = 'https://drive.google.com/file/d/1O6Nyg8yI4Kb-VaOlZr33_zkDGBDMrfwg/view?usp=drive_link'
url ='https://drive.google.com/uc?id=' + url.split('/')[-2]

train_df = pd.read_csv(url)

url = 'https://drive.google.com/file/d/1EeIIXUOEQDaHvTDGGvpa61BNVe-ggJIV/view?usp=drive_link'
url ='https://drive.google.com/uc?id=' + url.split('/')[-2]

valid_df = pd.read_csv(url)

url = 'https://drive.google.com/file/d/1Yh4CMGWT-ERZ6T7Q7JPVf2ZjGZx9Sh_O/view?usp=drive_link'
url ='https://drive.google.com/uc?id=' + url.split('/')[-2]

test_df = pd.read_csv(url)

In [4]:
train_df['reviews'] = train_df['reviews'].astype(str)
valid_df['reviews'] = valid_df['reviews'].astype(str)
test_df['reviews'] = test_df['reviews'].astype(str)

In [5]:
url = 'https://drive.google.com/file/d/1-wpLUWpIVBTr0BLHG79cZhHrZ34KFOJs/view?usp=drive_link'
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
data_df = pd.read_csv(url)
data_df['reviews'] = data_df['reviews'].astype(str)

In [6]:
data_df.columns = train_df.columns

train_data_df, valid_data_df = train_test_split(data_df, train_size=4000, random_state=42)

train_df = pd.concat([train_df, train_data_df], ignore_index=True)
valid_df = pd.concat([valid_df, valid_data_df], ignore_index=True)

print(f"Length of training data: {len(train_df)}")
print(f"Length of validation data: {len(valid_df)}")
print(f"Length of test data: {len(test_df)}")

Length of training data: 13140
Length of validation data: 1402
Length of test data: 463


In [7]:
train_df

,reviews,negative,neutral,positive
0,warung dimiliki pengusaha pabrik puluhan terke...,False,False,True
1,lokasi strategis jalan sumatera bandung nya ny...,False,False,True
2,betapa bahagia nya unboxing paket barang nya b...,False,False,True
3,makanan beragam harga makanan food stall kasir...,False,False,True
4,pakai kartu kredit bca tidak untung rugi besar,True,False,False
...,...,...,...,...
13135,masker camille emang bagus woii lahh emang mas...,False,False,True
13136,ready kah,False,True,False
13137,emang langsung putih tpi jgn coba kulit badak ...,False,False,True
13138,orang jateng cocok bgt manissss,False,False,True


In [8]:
# kamus_alay_df = pd.read_csv('https://raw.githubusercontent.com/nasalsabila/kamus-alay/refs/heads/master/colloquial-indonesian-lexicon.csv')
# kamus_alay_df

In [9]:
# kamus_alay_dict = kamus_alay_df.set_index('slang')['formal'].to_dict()

In [10]:
# def df_to_dataset(df, shuffle=True, batch_size=32):
#     """
#     Converts a pandas DataFrame into a TensorFlow Dataset.
#     - `reviews` is the text input.
#     - Target columns are `negative`, `neutral`, and `positive`.
#     """
#     features = df['reviews']  # Input text
#     labels = df[['negative', 'neutral', 'positive']]  # Multi-class targets
#     ds = tf.data.Dataset.from_tensor_slices((features, labels))
#     if shuffle:
#         ds = ds.shuffle(buffer_size=len(df))
#     ds = ds.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
#     return ds

# # Convert the datasets
# batch_size = 32
# train_ds = df_to_dataset(train_df, shuffle=True, batch_size=batch_size)
# valid_ds = df_to_dataset(valid_df, shuffle=False, batch_size=batch_size)
# test_ds = df_to_dataset(test_df, shuffle=False, batch_size=batch_size)

In [11]:
# def preprocess(reviews, label):
#     """
#     Preprocess the reviews using kamus_alay_dict.
#     Replace words if found in the dictionary; otherwise, keep the original word.
#     """
#     # Split the reviews into words
#     words = tf.strings.split(reviews)

#     # Map the dictionary lookup to each word
#     def replace_word(word):
#         # Use a Python dictionary lookup wrapped in tf.py_function
#         def lookup(w):
#             return kamus_alay_dict.get(w, w)  # Replace with dict value or keep the same
        
#         # Apply the lookup function and return
#         return tf.py_function(func=lookup, inp=[word], Tout=tf.string)

#     # Replace each word using map_fn
#     words = tf.map_fn(replace_word, words, fn_output_signature=tf.string)

#     # Join words back into a string
#     processed_review = tf.strings.reduce_join(words, separator=" ")

#     return processed_review, label



# train_ds = train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
# valid_ds = valid_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
# test_ds = test_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("tokenizer")

In [13]:
# import tensorflow as tf

# def tokenize(text):
#     # Tokenize the text using the tokenizer
#     return tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="tf")['input_ids']

# # Apply tokenization to each review
# X_train_tokenized = tf.convert_to_tensor(
#     [tokenize(text) for text in train_df['reviews']],
#     dtype=tf.int32
# )

# X_valid_tokenized = tf.convert_to_tensor(
#     [tokenize(text) for text in valid_df['reviews']],
#     dtype=tf.int32
# )

# X_test_tokenized = tf.convert_to_tensor(
#     [tokenize(text) for text in test_df['reviews']],
#     dtype=tf.int32
# )

# # Convert labels to tensors
# y_train = tf.convert_to_tensor(train_df.iloc[:, 1:].values, dtype=tf.int32)
# y_valid = tf.convert_to_tensor(valid_df.iloc[:, 1:].values, dtype=tf.int32)
# y_test = tf.convert_to_tensor(test_df.iloc[:, 1:].values, dtype=tf.int32)

# X_train_tokenized = tf.reshape(X_train_tokenized, (-1, 128))
# X_valid_tokenized = tf.reshape(X_valid_tokenized, (-1, 128))
# X_test_tokenized = tf.reshape(X_test_tokenized, (-1, 128))

# # Check the shapes of the tokenized datasets

# print(X_train_tokenized.shape)
# print(X_valid_tokenized.shape)
# print(X_test_tokenized.shape)

def tokenize(text):
    return tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="tf")['input_ids']

X_train_tokenized = tf.squeeze(tf.stack(train_df['reviews'].map(lambda x: tokenize(x)).tolist()), axis=1)
X_valid_tokenized = tf.squeeze(tf.stack(valid_df['reviews'].map(lambda x: tokenize(x)).tolist()), axis=1)
X_test_tokenized = tf.squeeze(tf.stack(test_df['reviews'].map(lambda x: tokenize(x)).tolist()), axis=1)

y_train = tf.convert_to_tensor(train_df.iloc[1:, 1:].values)
y_valid = tf.convert_to_tensor(valid_df.iloc[:, 1:].values)
y_test = tf.convert_to_tensor(test_df.iloc[:, 1:].values)

In [14]:
predictions = model.predict(X_test_tokenized)
predictions

15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 166ms/step


array([[ 2.718809  , -5.3317833 ,  0.3994115 ],
       [ 2.76671   , -4.195007  , -0.4184649 ],
       [ 3.2485332 , -2.509388  , -2.4076605 ],
       ...,
       [-1.1843995 ,  1.0652477 ,  0.67548263],
       [-1.2078623 ,  0.06133045,  1.4901582 ],
       [-0.02313354, -0.8224718 ,  0.96673864]], dtype=float32)

In [15]:
import heapq

def get_top_3_positive_index(predictions):
    positive_preds = predictions[:, 2]
    pq = []
    for i, pred in enumerate(positive_preds):
        if np.argmax(predictions[i]) == 2:
            heapq.heappush(pq, (-pred, i))
    top_3 = []
    for _ in range(3):
        if pq:
            _, task = heapq.heappop(pq)
            top_3.append(task)
    return top_3

def get_top_3_negative_index(predictions):
    negative_preds = predictions[:, 0]
    pq = []
    for i, pred in enumerate(negative_preds):
        if np.argmax(predictions[i]) == 0:
            heapq.heappush(pq, (-pred, i))
    top_3 = []
    for _ in range(3):
        if pq:
            _, task = heapq.heappop(pq)
            top_3.append(task)
    return top_3


In [16]:
def indices_to_texts(indices, data):
    texts = []
    for index in indices:
        texts.append(data[index])
    return texts

def get_top_3_positive_comments(predictions, data):
    top_3_positive_idx = get_top_3_positive_index(predictions)
    comments = indices_to_texts(top_3_positive_idx, data)

    return comments

def get_top_3_negative_comments(predictions, data):
    top_3_negative_idx = get_top_3_negative_index(predictions)
    comments = indices_to_texts(top_3_negative_idx, data)

    return comments

In [17]:
valid_predictions = model.predict(X_valid_tokenized)
print(get_top_3_negative_comments(valid_predictions, valid_df['reviews']))
print(get_top_3_positive_comments(valid_predictions, valid_df['reviews']))

44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 114ms/step
['woi orangorang kampung khusus nya daerah kalimantan barat bakar ladang kebon apalah pikir dampak nya ganggu penerbangan tolol babi bangsat mata dibuka woi orang kampung', 'ustad bajingan ustadtengkuzul kampanye sara kritik berita hoax si bajingan hadits palsu ceramah rokok kencing iblis', 'murahan opini sampah diusung pendukung korup']
['bandung lewatkan ngafe valey dago pakar menikmati makan malam menikmati pemandangan bagus makanan nya enak coba menginap hotel nya berbentuk caravan asyik banget udara nya segar sarapan nya mantap linting capgomeh nya enak banget', 'salah favorit keluarga lumayan datangi menu beragam pilihan bento ramen toast sushi nya oke interior ruang cukup nyaman bersantai bareng keluarga pelayanan cukup ramah harga terjangkau mantap', 'suasana resto cukup nyaman layanan cukup baik menawarkan beragam menu bahan dasar seafood udang gurame bawal kakap kepiting ayam sayuran gurame goreng gurih renyah sambal mangga nya enak ni

In [18]:
get_top_3_negative_comments(predictions, test_df['reviews'])

['gue pesimis manusia ngurusin sampah hewanhewan laut korban garagara buang sampah laut tidak tega gue liat nya kadang tidak mengerti pemimpinpemimpin mikirin sih urusan penting kayak gini he',
 'kecewa pelayanan hotel semalam tempati kamar mandi kotor pelayan tidak ramah',
 'rakyat indonesia kecewa settingan kayak sinetron asli rakyat indonesia kecewa tidak nonton asian games sctv besar rakyat indonesia india sinetron tidak jelas']

In [19]:
get_top_3_positive_comments(predictions, test_df['reviews'])

['menginap malam keluarga hotel nya bagus bersih fasilitas hotel lengkap breakfast nya enakenak suasana hotel sejuk nyaman cocok pilihan menginap keluarga',
 'setia traveloka pilihan maskapai nya payment nya mudah selaint harga nya murah mantap',
 'memakai berenam memakai kamar kamar besar bersih breakfast nya enak varian nya aneka segar juice pelayanan baik pelayanan restoran memuaskan rekomendasi']

In [20]:
test_df['reviews'][22]

'kecewa aplikasi hubungi pusat panggilan cek pemesanan bayar tidak muncul e tiket nya kirim bukti transaksi nya email tidak pusat panggilan menghubungi kecewa'

In [21]:
def get_prediction_class(predictions):
    m, n = predictions.shape
    y_pred_class = np.zeros((m, n))
    for i in range(m):
        max_val = 1e-3
        max_j = -1
        for j in range(n):
            if max_val < predictions[i, j]:
                max_val = predictions[i, j]
                max_j = j
        y_pred_class[i, max_j] = 1
    y_pred_class = y_pred_class.astype(bool)
    y_pred_class = np.argwhere(y_pred_class)[:, 1]
    return y_pred_class

In [22]:
predictions_class = get_prediction_class(predictions)
predictions_class

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 2, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 1, 1, 2, 1, 1, 0, 2, 1, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,

In [23]:
max_pred = np.max(predictions, axis=1)
# max_pred

### Using stanza

In [24]:
import stanza
from collections import Counter

# Initialize the Indonesian pipeline
custom_dir = "./stanza_models"

nlp = stanza.Pipeline('id', dir=custom_dir)

2024-11-25 16:44:17 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-11-25 16:44:18 INFO: Downloaded file to ./stanza_models\resources.json
INFO:stanza:Downloaded file to ./stanza_models\resources.json
2024-11-25 16:44:20 INFO: Loading these models for language: id (Indonesian):
| Processor    | Package      |
-------------------------------
| tokenize     | gsd          |
| mwt          | gsd          |
| pos          | gsd_charlm   |
| lemma        | gsd_nocharlm |
| constituency | icon_charlm  |
| depparse     | gsd_charlm   |

INFO:stanza:Loading these models for language: id (Indonesian):
| Processor    | Package      |
-----------------

In [25]:
# Sample sentences and one-hot encoded predictions
texts = ["Saya suka produk ini", "Ini sangat buruk", "Pelayanan yang luar biasa!", "Pembelian ini biasa saja"]
predictions = np.array([
    [0, 0, 1],  # Positive
    [1, 0, 0],  # Negative
    [0, 0, 1],  # Positive
    [0, 1, 0],  # Neutral
])

class_labels = np.argmax(predictions, axis=1)

word_counter = Counter()

for text, label in zip(texts, class_labels):
    if label == 2:  # Positive sentiment
        doc = nlp(text)
        
        # Extract nouns (objects)
        for sent in doc.sentences:
            for word in sent.words:
                if word.upos == "NOUN":  # Check if the word is a noun
                    word_counter[word.text.lower()] += 1

# Most common nouns/objects in positive predictions
most_common_words = word_counter.most_common()
print("Most common nouns/objects in positive predictions:", most_common_words)


Most common nouns/objects in positive predictions: [('produk', 1), ('pelayanan', 1)]


In [26]:
predictions = model.predict(X_test_tokenized)
texts = test_df['reviews'].values

15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 110ms/step


In [27]:
custom_dir = "./stanza_models"

nlp = stanza.Pipeline('id', dir=custom_dir)

2024-11-25 16:44:33 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-11-25 16:44:34 INFO: Downloaded file to ./stanza_models\resources.json
INFO:stanza:Downloaded file to ./stanza_models\resources.json
2024-11-25 16:44:36 INFO: Loading these models for language: id (Indonesian):
| Processor    | Package      |
-------------------------------
| tokenize     | gsd          |
| mwt          | gsd          |
| pos          | gsd_charlm   |
| lemma        | gsd_nocharlm |
| constituency | icon_charlm  |
| depparse     | gsd_charlm   |

INFO:stanza:Loading these models for language: id (Indonesian):
| Processor    | Package      |
-----------------

In [28]:
# Bedaaa
import re
def replace_emoji_with_word(text):
    for emoji, word in emoji_dict.items():
        text = re.sub(f'({emoji})+', f' {word} ', text)
    return text.strip()

def stop_words_removal(text):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    removed_stop_words = [word.lower() for word in tokens if word.lower() not in stop_words]
    return ' '.join(removed_stop_words)

def preprocess_text(text):
    text = text.lower()
    text = replace_emoji_with_word(text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.strip()
    text = stop_words_removal(text)
    return text

def tokenize_batch(texts):
    return tokenizer(
        texts, padding="max_length", truncation=True, max_length=128, return_tensors="tf"
    )['input_ids']

# PREDICT
def predict_sentiment_batch(texts):
    # preprocessed_texts = [preprocess_text(text) for text in texts]
    tokenized_texts = tokenize_batch(texts)
    predictions = model.predict(tokenized_texts)
    sentiment_labels = ["Negatif", "Netral", "Positif"]
    sentiments = np.argmax(predictions, axis=1)
    return sentiments

In [29]:
# predictions = predict_sentiment_batch(test_df['reviews'])
# predictions

In [30]:
# The texts to predict
texts = ['kembalikan philly cheese steak min',
       'halo @dominos_id aku mau COMPLAIN ya... pesen lewat aplikasi wa, data uda sukses pemesanan take away karena buru2... daaaaan sampe store pesenanku ga ada dooong... orang store toko bilang ga ada masuk, sedangkan aku uda pesen via wa yg tercantum disini, kecewa bgtt ai parah, niat buru2 biar lebih simple, aplikasi ga bisa dipake mesen juga bolak balik donlot ulang tetep aja aplikasi error, tolooooong yaaaa... pelayanan buruk, bad bad bad',
       'Sayangnya pakai app tinggal klik kirim ke alamat ga bisa2 bahkan udh di outlet nya tetep ga bisa ya sudah nyerah dech',
       'Papi duo masih 57rban min ?',
       'App lebih murah jangan mau datanh ke oulet dah lebih mahal ngantri pula kena zonk 😂😂😂',
       'Min beda ht, tc, crunchy, pan, hm aplgi ya? Btw itu gmn si apa',
       'Ya sy baru nyoba enaknya mmg 👍👍👍ga cuman iklan sj tp top enak',
       '@dominos_id okk min trimakasih',
       '@dominos_id emang pengantaran 5menit jarak nya terlalu jauh ya? Emang terjauh jarak nya berapa menit',
       'Kak tolong cek dm ya', 'Baru nyoba kemarin, beneran enak 🤤☺️',
       'Pelayanan @dominos_id mojokerto sangat sangat buruk',
       'Fav banget pizza cheese kaya gini 😍🤤',
       'Izin mau kasi saran...  Untuk store yg di pematang siantar tolong bilang ke karyawan nya agar lebih higienis lagi dalam melakukan pekerjaan..  Habis hitung uang sebaiknya cuci tangan dulu untuk mengerjakan pizza yg mau di panggang..  Sudahlah tidak pakai sarung tangan..  Langsung di depan mata lihat begitu jadi iuh... Mual @dominos_id',
       '@dominos_id oh ini ada min struk nya, saya kirim dr DM yah.. Baru saya cr di tas masih ada',
       'Sumpah enak banget loh yg rasa ini 😍',
       'lama banget pesanannya jadi 😢', 'Bangkeee',
       'Kecewa..perdana rasain krain enk bgt ..beli di mobil dbireuen yg bakal buka dilhokseumawe aceh, udh bli nya gk sesuai dksi..',
       'Min cek dm saya beli pizza GK ada saos nya',
       'Mau tanya apakah peraturan skrg klo mau saos sambal kena charge Rp 500/sachet?',
       'Siang @dominos_id berapa hari yang lalu saya ada beli Pizza 2 box.... Pizza nya bantet adonan nya dan dalam nya kering.... Saya sangat kecewa. Saya ada Foto Pizza nya.',
       '@dominos_id saya ada dm...silahkan di lihat foto nya....',
       '@dominos_id uk brp kak', 'AYE DOMINO',
       'kok ga ada menu nya min di aplikasi? di papi duo promo juga gak ada',
       'Sama ini is @istiqomahbalya', "Wa domino's gaada ya",
       'Kenapa ngk buka d pangkalan kerinci dih jauh bnget mau beli ke pekan 😢😢',
       'Promo scan and win bohongan sudah dua kali beli domino ga bs dscan bilangnya false terus',
       '@romaarmdhn15 🙏🙏',
       'Kak aplikasi dominos pizza lg error kah kok aku gabisa dpt kode otp drtd 😭',
       '@sriellagisti27 pdhl pangkalan kerinci teramat ramai ya kak😂 gpp ke pku aja beli nya, cuma 4jam-an kok kak😂',
       'Aplikasinya sampah bgt si. Gabisa dibuka dr pagi buset',
       'plis buka cabang di pontianak😢',
       '@dominos_id buka di KalBar dong 🙏🥺',
       'Admin buat tutorial donk, gmna caranya makan pizza medium dgn 6 potong irisan tapi saosnya cuman dikasih 2 😂😂',
       'Buka cabang di banyuwangi jatim dong kak',
       'Parah banget resto jakarta pusat ga bisa bantu cancel padahal rumah saya di Citra Raya Tangerang...habis aku di omelin Drive',
       'bisa beli langsung?',
       'Ini gimana ya saya pesen dominos sawangan, status nya belum terkirim malah udah ke kirim status nya dan orderan udah selesai, disuruh kontak ke dominos nya, mereka bilang ga ada pesanan, repot banget, refund juga ribet. Pelayanannya gimana sih? Malah dilempar2, buang waktu tenaga @dominos_id @grabfoodid @grabid',
       'Bikin Domino X Keju Recheeseeeee', 'Buka di Cilacap dong min',
       'Enak sih tapi gak Kya di video ini 😢', '@dominos_id gak di read',
       'Skr kan lagi hut bank CIMB niaga yg mana salah merchant yg di infokan utk dapat cashback adl dominos, tapi harus pake qris via bank mandiri, nah tadi pihak kasir alesan mulu alat dr mandiri ga bisa, krna dia tau klo pake itu bs dapat diskon lagi, gmn siy tu kasir',
       'Sedih banget american all star nya udah gak ada.. padahal itu yang paling favorite! Menunya baru semua ya dominos???? Please kembalikan american all star ku 😢😢😢😢😢',
       'Promo Oktober mana min?',
       '@dominos_id kak itu varian mac n chesee gitu ya',
       'Payahh resto 1 @dominos_id ini ngk ada kmr mandi nya.... Yg ngalamin hal samarinda kebelet pipis tpi bungkusan kita blm siap...........',
       'Ini size medium apa reguler?',
       'Ka domino pizza ada paket ulang tahun ga?', 'cek DM please',
       'Atau yg ini ty @syifa_rfh',
       '@dominos_id surprise in yg ultah aja ka tapi gak ngadain acara, klo di pi** kan ada',
       '@dominos_id tnks kk, Spil minumanya kk yg enak',
       '@dominos_id Spil minumanya kk', '@dominos_id iya kk',
       '@dominos_id kalo beli 1 doang brapa ka itu',
       'min, saya udh download aplikasi dominos. tp gak msk2 email verifikasinya di email',
       '@dominos_id sudah saya email kak',
       '@dominos_id good service 👍… saya langsung di telp oleh CS nya dan solved',
       'Buat ank ku yg bentar lagi ulang tahun 😍',
       '@dominos_id papi duo promo sampai kapan kak',
       'Promonya masih ada min bulan oktober ini',
       'Mau tanya dong ini outletnya buka jambrpa ya?',
       '@dominos_id apa ka promonya', 'Ini harga perpizza brp yaa?',
       'Min, nama pizza yg Viral dulu apa min? Pengen cobain tapi ngga tau namanya 😢',
       'Bilang nya gmna min kalo mau promo itu ? Dismua gerai bisa kah ?',
       '@jamal_alfkiir bilang nya mau beli promo papi duo?',
       'Ultimate cheese ini enak menurutku', 'Promonya sampai kapan ka?',
       '@dominos_id oke mksh infonya ka', 'Itu 3 brp min?',
       'Ultimate Cheese', 'Ultimate cheese pizza',
       'Tp punya ku td malam beli ga semelimpah itu😢',
       "please @dominos_id improve your service at your kalibata city's outlet!!!",
       'Berapa itu satuan',
       '@dominos_id wawww seru kalau makan bareng keluarga😮',
       'Papi trio yg mana ka',
       '@dominos_id coba yg no 2 beda sm yg ada di video',
       'American pie biasa aja 😂', 'Apa ini ga ada volcanoooo',
       '@dominos_id tuna delight ini beneran discontinue apa nggak ada sementara aja? 😞😞😞😞 pizza favorit padahal',
       'Brp hrg',
       '@dominos_id kenapa ga ada min padahal enakan tuna deligjt loh',
       'Papi trio masih ada?',
       'Spreme cheese pie ini pav anakku tolong @dominos_id jgn ilangin menu ini.. anakku gtm anakku dirawat di RS maunya cuma makan domino supreme cheese aja',
       'No 1 dan no 3 setujuuu👏👏👏👏👏', '@henhenhenhens enakan mana kak?',
       '@dominos_id bedanya ultimate cheese melt dan pie supreme cheese apa kak?',
       '@verenaprilianii klo suka kejuuuuu banget, saran coba yg ultimate cheese',
       'min papi viral gada lagi ya?', 'Berapa ?',
       'Min papi duo masi berlaku?',
       '@dominos_id ini ukurannya seberapa min??',
       '@dominos_ ini ukuran seberapa min??',
       'Masih ada engk promo pizza yg ini min?',
       '@dominos_id cusss pesen kakak😍😍',
       'Udah semua yg paling fav bgt yg nomor 2 OMG enak bgttt udah sering bgt pesan itu❤️❤️❤️',
       'Brapa ya kak', '@dominos_id hari ini masih promo ga kak ?',
       'Apa promo masih ada',
       'Kalau makan di tempat gk habis bisa take way gk kk.?',
       'Papi duo medium cuma dapet dua pizza. Ada paketnya g min yang isinya tiga varian ini ?',
       '@dominos_id kalau ketigah tigahnya berapa kk',
       'Kalau ketigah tigahnya berapa kk.?', 'Brp an kak', 'Enak semuaa🔥',
       'Ga ada promo beli 2 dg potongan harga ya?',
       '@dominos_id papi duo rasa pizza nya bisa milih atau gimana?',
       'Spil harganya kk', '@dominos_id tnks kk',
       '@dominos_id papi duo medium itu apa', 'So yummy',
       'ih yang ini nahh 85 ribu sih 😆😢 @lillah1898',
       '@hannalillah18 gasss 🤭',
       'Enakk banget .. aku yang gak doyan pizza aja doyan banget sama pizza domino chesse nya enak gak bikin eneg .. 😍',
       'Please yg ini berapaan?',
       '@alineoktaviane wah terimakasih min infonya, pulang kerja cuss ke dominos',
       'Kok pengen @trippinjann',
       '@dikaisn iyh domino sama pizzahut juga sama',
       'Aku psen yang ini yang large bner kaya roti biasa topingnya dikit bgt...',
       'Beda dengan yang medium bnyak kejunya...knpa bisa sperti itu ya???? Saya mrasa rugi aja gt',
       'Ticket number 10589380... not resolved yet',
       'Yang 3 pizza pinggir nya ada kejunya itu apa ya nama pizza nya dl pernah pernah pesen cuma lupa namanya',
       '@dominos_id skrng sudah tidak ada ya kok saya liat di app tidak ada',
       '@dominos_id kon di apo gk ada ya skrng pizza itu cheese burst dulu saya beli pizza yg dapet 3 itu',
       '@ratnarendiany gass teh ngiler 🤤', '@eka_puspita hayuuu gas ken',
       '@ratnarendiany gaasss',
       'Kmren beli langsung di outletnya 90 rebuan, ko di shope food cuma 50 rebu . Itu bedanya apa yak, beda harganya jauh bgt',
       'kira kira crust yang cocok apa buat ultimate cheese melt',
       '@dominos_id kalo ultimate meat overload sama volcano cheese crust nya ht juga min cocok nya?',
       'Wenakkk rekkkk',
       '@dominos_id tadi baru aja beli,enakk bangett nagih❤️❤️🔥',
       'Chease',
       'beda dari aslinya😂 udah beli malah kejunya dikit banget jadi nyesel beli nyaa',
       '@dominos_id di dm ga dibalass', 'Paling suka yg ini😍',
       '😢❤️🔥🙌👏😍😮😂😂😂😂',
       'INI ENAK BGT SUMPAH SAMA YG VOLCANO JG ENAK BGT YA ALLAH PGN NANGISSS MAU LAGI BSK MAU BELI LAGIIIII😭😭😭😭❤️❤️❤️❤️❤️❤️',
       'Gara² menu ini seminggu bs 3x pesen. Efeksamping BB naikk 😬',
       '@fatricia_budiaty huuuu 😭😭😭',
       '@rika_yanti789 iya kmren w jg beli tp kejunya ga luber kya di iklan. Yg di iklan itumah kejunya udh tambahan jdi harganya jg nambah',
       'Sangat kecewa pizzanya tiba sudah dingin, klaimnya "lebih dari 30menit Free" di Apps nya tapi tidak terbukti, dan DM di IG inipun sudah 1 minggu lebih tidak ada respon',
       'Saya order melalui app, dari azan isya sampe jam 9 blm sampai pesanan status di app delivery,',
       'Pesanan di dominos pos pengumben',
       '@seravinanathania aku dapetnya kering😢apa Krn udah dingin ya',
       'Kalo di video beda sama di realita.. mozanya gak sebanyak ituu',
       'Favorite banget',
       '@dominos_id dimana vouchernya gada voucher sama sekali',
       'Ini masih', 'Favorit sih ini enak bgt',
       '@@dominos_id di online kaya grab dan gofood, ultimate cheese ada yg 83,85 dan 103rb beda nya apa?',
       '@ajadspk_ pgnnnn iniiiiii',
       'Saya pesan via aplikasi nya, tp gak dikasih struk nya sm bang kurirnya.. Pdhl kan mau ikutan event nya.. Parah sih',
       'Fav aku nih', 'Enak bgt kemaren udah ke kelapa gading 😍',
       'Ada porsi kecil ga ?pengen tapi sendirian', 'Brp harganya',
       '𝖡𝗂𝗌𝖺 𝗅𝖾𝗐𝖺𝗍 𝗀𝗋𝖺𝖻 𝖿𝗈𝗈𝖽 𝗀𝗄 𝗄𝖺𝗄', 'Enakk parahh',
       '@dominos_id promo papi duo pembelian dimana kak',
       '@dominos_id nggk bisa sama skali', 'Harga brp stlh diskon min?',
       'Jd kena brp min?', 'min cek dm dong', 'Kepengen 🤤@hafiznafilah',
       '@dominos_id sampai kapan ?', 'Terbaeekkk nih favorit aku 🤩🤩🤩',
       'La ini tu promosi yang gak bikin ribet 😍😍',
       '@dominos_id dpt brp box pitza kak dan varian ap yaaa',
       'Kalo lewat app cara mesennya gimana sih min',
       'Pesan gimana biar sama persis gtu? Soalnya banyak yg komen isiannya gk sesuai 😂',
       '@dominos_id kalau yg promo papi duo itu persis ky gtu gk min?',
       'Masih discon gak?', '@dominos_id diskon smp kapan kak',
       'Yg di video itu pizza apa aja ya min?',
       'Masih promosi gak yaang pie supreme cheese min @dominos_id',
       '@dominos_id untuk pembelian 1 atau 3 min', 'Berapaan min?',
       '@dominos_id sampai kapan KK promonya?',
       '60rb itu udh diskon 30%?',
       'Ka bisa delivery ga ? Atau harus takeaway lewat apk',
       '@dominos_id tapi tapi tapiii saya beli yang 100an kena ongkir min..',
       '@dominos_id Applikasi domino nya kak min', 'pizza kecil itu',
       'Sampai kapan min promonya', 'Mauuu paket nya apa nama nya..kk',
       'Di jember ada gak?', 'Bln niv masih berlaku ga',
       'Gimna cara ordernya kk', 'Sampe tanggal berapa min?',
       '@valerian_evander mending ini nyet',
       'saya udh download tpi gak bisa ka..', 'Sudah sama pajak kk?',
       '@dominos_id jadi brpa kk', '@halidddd_ emg lu doyan pizza?',
       'Di ambil sendri ke store atau delivery y min?',
       'Makasih domino udah ciptain paket ini,berkat paket ini aku bisa teraktir teman 1 team ku karena murce tapi keliatan wah,edisi ulta akoh 😍😍',
       'Beli ini syg @dhy.sn12', '@dominos_id oke kak, thanks yaaa☺☺',
       'ultimate cheese ini enak menurutku'
       '😮1kotak pizza 4potong.. maksudnya 5kotak gitu', 'Larange']

preprocessed_texts = [preprocess_text(text) for text in texts]
class_labels = predict_sentiment_batch(preprocessed_texts)

# class_labels = np.argmax(predictions, axis=1)

exclude_words = ['gua', 'kak', 'gue', 'sih', 'kasih', 'banget', 'orang', 'bu', 'sumpah', 'gitu', 'bnyak', 'banyak', 'gt', 'gitu', 'duo', 'dua', 'satu', 'min', 'pesen', 'brp', 'berapa','memang', 'mmg', 'udh', 'udah', 'uda', 'niat', 'tp', 'tapi', 'menit']

pos_counter = Counter()
neg_counter = Counter()

for text, label in zip(preprocessed_texts, class_labels):
    # Positive
    if label == 2:  
        doc = nlp(text)
        for sent in doc.sentences:
            for word in sent.words:
                # Check if the word is a noun or in the custom noun dictionary
                word_text = word.text
                if word_text in exclude_words:
                    previous_noun = None
                    continue
                if (word.upos == "NOUN" or word_text in nouns):
                    previous_noun = word_text
                # Check for adjective and combine with previous noun
                elif (word.upos == "ADJ" or word_text in adjectives) and previous_noun:
                    phrase = f"{previous_noun} {word_text}"
                    pos_counter[phrase] += 1
                    previous_noun = None  # Reset previous noun after pairing
                # if word.upos == "ADJ" and word.head > 0:
                #     head_word = sent.words[word.head - 1]
                #     if head_word.upos == "NOUN" and head_word.text.lower() not in exclude_words:
                #         phrase = f"{head_word.text.lower()} {word.text.lower()}"
                #         pos_counter[phrase] += 1
                # if word.upos == "NOUN" and word.text.lower() not in exclude_words: 
                #     pos_counter[word.text.lower()] += 1
    # Negative
    elif label == 0:
        doc = nlp(text)
        for sent in doc.sentences:
            for word in sent.words:
                word_text = word.text
                if word_text in exclude_words:
                    previous_noun = None
                    continue
                if word.upos == "NOUN" or word_text in nouns:
                    previous_noun = word_text
                # Check for adjective and combine with previous noun
                elif (word.upos == "ADJ" or word_text in adjectives) and previous_noun:
                    phrase = f"{previous_noun} {word_text}"
                    neg_counter[phrase] += 1
                    previous_noun = None  # Reset previous noun after pairing
                # if word.upos == "ADJ" and word.head > 0:
                #     head_word = sent.words[word.head - 1]
                #     if head_word.upos == "NOUN" and head_word.text.lower() not in exclude_words:
                #         phrase = f"{head_word.text.lower()} {word.text.lower()}"
                #         neg_counter[phrase] += 1
                # if word.upos == "NOUN" and word.text.lower() not in exclude_words:  
                #     neg_counter[word.text.lower()] += 1
pos_common_words = pos_counter.most_common()
neg_common_words = neg_counter.most_common()
print("Yang disukai customer:", pos_common_words)
print("Yang tidak disukai customer:", neg_common_words)

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step
Yang disukai customer: [('cheese enak', 2), ('cheese kaya', 1), ('false sopan', 1), ('raya langsung', 1), ('grabid enak', 1), ('video sedih', 1), ('email good', 1), ('service bagus', 1), ('wawww seru', 1), ('keluarga kaget', 1), ('setujuuu bagus', 1), ('omg enak', 1), ('promo enak', 1), ('chesse enak', 1), ('crust cocok', 1), ('nagih keren', 1), ('nyesel senyum', 1), ('cinta sedih', 1), ('apresiasi bagus', 1), ('kaget enak', 1), ('voucher favorit', 1), ('nih enak', 1), ('promosi ribet', 1), ('nama nyakk', 1)]
Yang tidak disukai customer: [('ai parah', 1), ('pelayanan buruk', 1), ('dech murah', 1), ('oulet mahal', 1), ('jarak jauh', 1), ('mojokerto buruk', 1), ('karyawan higienis', 1), ('pekerjaan habis', 1), ('tangan langsung', 1), ('cinta lama', 1), ('pesanan sedih', 1), ('adonan kering', 1), ('dih jauh', 1), ('pekan sedih', 1), ('pontianak sedih', 1), ('saosnya parah', 1), ('pesanan repot', 1), ('refund ribet', 1), ('niaga salah', 1), ('bs disko

In [31]:
from collections import Counter

def analyze_sentiment(texts):
    # Preprocess texts and predict sentiment
    preprocessed_texts = [preprocess_text(text) for text in texts]
    class_labels = predict_sentiment_batch(preprocessed_texts)
    
    # Initialize counters for positive and negative sentiment words
    pos_counter = Counter()
    neg_counter = Counter()

    # Analyze the texts based on sentiment labels
    for text, label in zip(preprocessed_texts, class_labels):
        previous_noun = None
        doc = nlp(text)
        
        for sent in doc.sentences:
            for word in sent.words:
                word_text = word.text
                
                # Skip words in exclude_words list
                if word_text in exclude_words:
                    previous_noun = None
                    continue
                
                # Check for noun or custom nouns
                if word.upos == "NOUN" or word_text in nouns:
                    previous_noun = word_text
                
                # Check for adjective and combine with previous noun
                elif (word.upos == "ADJ" or word_text in adjectives) and previous_noun:
                    phrase = f"{previous_noun} {word_text}"
                    if label == 2:  # Positive sentiment
                        pos_counter[phrase] += 1
                    elif label == 0:  # Negative sentiment
                        neg_counter[phrase] += 1
                    previous_noun = None  # Reset previous noun after pairing

    # Get the most common words
    pos_common_words = pos_counter.most_common()
    neg_common_words = neg_counter.most_common()

    # Print results
    # print("Yang disukai customer:", pos_common_words)
    # print("Yang tidak disukai customer:", neg_common_words)
    
    return pos_common_words, neg_common_words

def get_top_3_common_words(pos_common_words, neg_common_words):
    top_3_pos_words = [word for word, _ in pos_common_words[:3]]
    top_3_neg_words = [word for word, _ in neg_common_words[:3]]
    
    return top_3_pos_words, top_3_neg_words

def analyze_sentiment_top_3(texts):
    pos_common_words, neg_common_words = analyze_sentiment(texts)
    return get_top_3_common_words(pos_common_words, neg_common_words)

print(analyze_sentiment_top_3(texts))


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step
(['cheese enak', 'cheese kaya', 'video sedih'], ['ai parah', 'pelayanan buruk', 'oulet mahal'])


In [32]:
custom_noun_tags = {"cheese"}  # Add words you know are nouns but may not be detected.

def extract_adj_noun_phrases(texts):
    phrases_counter = Counter()

    for text in texts:
        doc = nlp(text)
        for sent in doc.sentences:
            previous_noun = None
            for word in sent.words:
                word_text = word.text.lower()
                # Check if the word is a noun or in the custom noun dictionary
                if word.upos == "NOUN" or word_text in nouns:
                    previous_noun = word_text
                # Check for adjective and combine with previous noun
                elif (word.upos == "ADJ" or word_text in adjectives) and previous_noun:
                    phrase = f"{previous_noun} {word_text}"
                    phrases_counter[phrase] += 1
                    previous_noun = None  # Reset previous noun after pairing

    return phrases_counter

In [36]:
texts = ['ultimate cheese ini nggak enak menurutku']
texts = [preprocess_text(text) for text in texts]
# texts
adj_noun_phrases = extract_adj_noun_phrases(texts)
print("Most common adjective-noun phrases:", adj_noun_phrases.most_common())

Most common adjective-noun phrases: []


In [34]:
# Example texts
texts = [
    'ultimate cheese enak menurutku',
    'makanan enak banget tai'
]

# Define custom words to exclude
exclude_words = {"tapi", "dan", "sangat"}

# Function to extract adjective-noun pairs
def extract_adj_noun_phrases(texts):
    phrases_counter = Counter()
    
    for text in texts:
        doc = nlp(text)
        for sent in doc.sentences:
            for word in sent.words:
                # Find adjective (ADJ) modifying a noun (NOUN)
                if word.upos == "ADJ" and word.head > 0:
                    head_word = sent.words[word.head - 1]
                    if head_word.upos == "NOUN" and head_word.text.lower() not in exclude_words:
                        phrase = f"{head_word.text.lower()} {word.text.lower()}"
                        phrases_counter[phrase] += 1

    return phrases_counter

# Get adjective-noun phrases
adj_noun_phrases = extract_adj_noun_phrases(texts)

# Print most common phrases
most_common_phrases = adj_noun_phrases.most_common()
print("Most common adjective-noun phrases:", most_common_phrases)

Most common adjective-noun phrases: [('makanan enak', 1)]
